# Creating and processing logs

## Goals
  
  - importance of logging
  - configuring logger module
  - open log files
  
## modules

In [ ]:
from __future__ import print_function

import logging
import logging.config
import logging.handlers
import mimetypes
import gzip, bz2
import yaml



## Importance of Logging

All automation and scripting activity should be carefully logged.

The ```logging``` module:

  - can stream logs to files and network
  - is configurable via yml 
  - optimizes output via log levels
  - takes care of rotation
  
Hint: manage logs with syslog.
  

In [ ]:

# Logs to stdout|err with a default loglevel.
# Logs are not printed on jupyter cells but you can  
#  check them in another terminal with
#  #docker-compose logs
logging.basicConfig(level=logging.DEBUG)

# Create a logger.
log = logging.getLogger()

# Logs supports a print-like syntax, and doesn't 
#  build unneeded message strings. 
log.info("Use %r instead of %s to avoid", [u"Serialization"], "issues")



In [ ]:

%cat logger.yml


In [ ]:
with open('logger.yml') as logger_config:
    logging.config.dictConfig(yaml.safe_load(logger_config))
    
# The ```os``` module goes to syslog
log.info("To syslog")


In [ ]:
# To process compressed files, use an helper function. 
import mimetools
import gzip
import bz2

def log_open(path):
    """Open log files using its mimetype to choose the correct method"""
    l_type, l_encoding = mimetypes.guess_type(path)
    if l_encoding == 'gzip':
        return gzip.open(path, 'rb')
    elif l_encoding == 'bzip2':
        return bz2.BZ2File(path, 'rb')
    else:
        return open(path, 'rthe b')


In [ ]:
# Exercise:
#  log some messages modifying the default format string.
#  check the outcome in a separate terminal with
#  docker-compose logs

## Parsing /proc

Linux /proc filesystem is a cool place to get data

In the next example we'll see how to get:
 - thread informations;
 - disk statistics;
 
 

In [ ]:
# Parsing /proc - 1
def linux_threads(pid):
    """Retrieving data from /proc
    """
    from glob import glob
    # glob emulates shell expansion of * and ?
    path = "/proc/{}/task/*/status".format(pid)
    
         
    # pick a set of fields to gather
    t_info = ('Pid', 'Tgid', 'voluntary')  # this is a tuple!
    for t in glob(path):
        # ... and use comprehension to get 
        # intersting data.
        t_info = [x 
                  for x in open(t) 
                  if x.startswith(t_info)] # startswith accepts tuples!
        print(t_info)

In [ ]:
# If you're on linux try linux_threads
pid_of_init = 1  # or systemd ?
linux_threads(pid_of_init)

In [ ]:
# Reload previous grep 
from course import grep
# On linux /proc/diskstats is the source of I/O infos
disk_l = grep("sda", "/proc/diskstats")
print(''.join(disk_l))

# To gather that data we put the header in a multiline string
from course import diskstats_headers as headers

#Take the 1st entry (sda), split the data...
disk_info = disk_l[0].split()
# ... and tie them with the header
ret = zip(headers, disk_info)

# On py3 we need to iterate over the generators
print(list(ret))

In [ ]:
# Try to mangle ret
print('\n'.join(str(x) for x in ret))

In [ ]:
# We can create a reusable commodity class with
from collections import namedtuple

# using the imported `headers` as attributes
# like the one provided by psutil
DiskStats = namedtuple('DiskStat', headers)

# ... and disk_info as values
dstat = DiskStats(*disk_info)
print(dstat.device, dstat.writes_ms)

# Homework: check further features with
# help(collections)

In [ ]:
# Exercise
# Write the following function 
def linux_diskstats(partition):
    """Print every second I/O information from /proc/diskstats
    
        @param: partition - eg sda1 or vdx1
        
        Hint: use the above `grep` function
        Hint: use zip, time.sleep, print() and *magic
    """
    diskstats_headers = ('reads reads_merged reads_sectors reads_ms'
            ' writes writes_merged writes_sectors writes_ms'
            ' io_in_progress io_ms_weight').split()
    
    while True:
        raise NotImplementedError
        print(values, sep="\t")

In [ ]:
# %load course/linux_diskstats.py
__author__ = 'rpolli'
from __future__ import print_function
from course import grep


def linux_diskstats(disk):
    """Get I/O information from /proc/diskstats

       @param disk def sda
       goal: usage of time.sleep
       goal: usage of dict.setdefault
       goal: use string concatenation to increase readability
       goal: use *magic with print+sep, splitting and slicing
    """
    from time import sleep
    info = ('reads reads_merged reads_sectors reads_ms'
            ' writes writes_merged writes_sectors writes_ms'
            ' io_in_progress io_ms_weight').split()
    print(*info, sep=",")
    old, cur = dict(), dict()
    while True:
        disk_l = grep(disk, "/proc/diskstats")
        for x in disk_l:
            info = x.split()
            # the first 3 fields are disk informations
            part = info[2]
            old.setdefault(part, [0] * 11)
            cur[part] = map(int, info[3:])
            delta = [x - y for x, y in zip(cur[part], old[part])]
            print(*delta, sep=",")
            old[part] = cur[part]
        sleep(1)


In [ ]:
# Using check_output with split() doesn't always work
from os import makedirs

makedirs('/tmp/course/b l a n k s')  # , exist_ok=True)

In [ ]:
from subprocess import check_output

check_output('ls "/tmp/course/b l a n k s"'.split())

In [ ]:
# You can use
from shlex import split
# and
cmd = split('dir -a "/tmp/course/b l a n k s"')
check_output(cmd)

In [ ]:
# Cleanup everything!
from shutil import rmtree
rmtree("/tmp/course")

## zip on py3 is a generator 


In [ ]:
# zip_iterables():
"""The zip method joins list elements pairwise
    like a zip fastener
"""
from sys import version_info as python_version
a_list = [0, 1, 2, 3]
b_list = ["a", "b", "c", "d"]
zipper = zip(a_list, b_list)
print(zipper)

In [ ]:
if python_version >= (3,):
    zipper = list(zipper)
assert zipper == [(0, "a"), (1, "b"), (2, "c"), (3, "d")]